**Finance 6320: Computational Finance** <br>
**Assignment #3** <br>
**Due: TBD** <br>



## Imports

In [1]:
import numpy as np
from scipy.stats import binom

**Problem 1.** 

In this problem you will use the classes that we have developed to price European puts and calls with the multiperiod Binomial Option Pricing Model (BOPM). This problem is more or less done for you in our lectures.

Demonstrate that this code works by replicating the pricing of the European put option in Figure 10.6 of the McDonald book on page 310. You should get a price of $\$2.999$ for the put option.

Also, price the corresponding call option to demonstrate the algorithm, implemented once, is polymorphic and is capable of pricing call options as well.

In [68]:
class VanillaOption(object):
    
    def __init__(self, strike, expiry):
        self.strike = strike
        self.expiry = expiry
    
    def payoff(self, spot):
        return payoff(spot, strike)
    
class CallOption(VanillaOption):
    
    def payoff(spot, strike):
        return np.maximum(spot - strike, 0.0)

class PutOption(VanillaOption):
    
    def payoff(self, spot):
        return np.maximum(self.strike - spot, 0.0)

#### Question:
When I use $spot-strike$, the code works, but when I use $self.strike$, it won't work(Compare the difference between call payoff and put payoff).

In [65]:
def EuropeanBinomPricer(CallOption, spot, rate, vol, div, steps):
    
    h = expiry / steps
    nodes = steps + 1
    u = np.exp((rate - div) * h + vol * np.sqrt(h))
    d = np.exp((rate - div) * h - vol * np.sqrt(h))
    pstar = (np.exp((rate - div) * h) - d) / (u - d)
    discount = np.exp(-(rate - div))
    callT = 0.0
    
    for i in range(nodes):
        spotT = spot * (u ** (steps - i) * (d ** i))
        callT += CallOption.payoff(spotT, strike) * binom.pmf(steps - i, steps, pstar)
    callprice = callT * discount
    
    
    return callprice

In [66]:
spot = 41.0
strike = 40.0
vol = 0.3
rate = 0.08
div = 0.0
expiry = 1.0
steps = 2

result = EuropeanBinomPricer(CallOption, spot, rate, vol, div, steps)
print("The price for European call option is:", result)

The price for European call option is: 7.1622842046


In [69]:
def EuropeanBinomPricer(VanillaOption, spot, rate, vol, div, steps):
    
    h = expiry / steps
    nodes = steps + 1
    u = np.exp((rate - div) * h + vol * np.sqrt(h))
    d = np.exp((rate - div) * h - vol * np.sqrt(h))
    pstar = (np.exp((rate - div) * h) - d) / (u - d)
    discount = np.exp(-(rate - div))
    putT = 0.0
    
    for i in range(nodes):
        spotT = spot * (u ** (steps - i) * (d ** i))
        putT += PutOption.payoff(spotT, strike) * binom.pmf(steps - i, steps, pstar)
    putprice = putT * discount
    
    
    return putprice

In [70]:
spot = 41.0
strike = 40.0
vol = 0.3
rate = 0.08
div = 0.0
expiry = 1.0
steps = 2

result1 = EuropeanBinomPricer(CallOption, spot, rate, vol, div, steps)
print("The price for European put option is:", result1)

AttributeError: 'numpy.float64' object has no attribute 'strike'

The object oriented is not working so I change the coding strategy in following way:

#### Here is the European Call Option:

In [83]:
def callpayoff(spot, strike):
    return np.maximum(spot - strike, 0.0)

def putpayoff(spot, strike):
    return np.maximum(strike - spot, 0.0)

In [81]:
def EuropeanBinomPricer(callpayoff, spot, rate, vol, div, expiry, steps):
    
    h = expiry / steps
    nodes = steps + 1
    u = np.exp((rate - div) * h + vol * np.sqrt(h))
    d = np.exp((rate - div) * h - vol * np.sqrt(h))
    pstar = (np.exp((rate - div) * h) - d) / (u - d)
    discount = np.exp(-(rate - div))
    callT = 0.0
    
    for i in range(nodes):
        spotT = spot * (u ** (steps - i) * (d ** i))
        callT += callpayoff(spotT, strike) * binom.pmf(steps - i, steps, pstar)
    callprice = callT * discount
    
    
    return callprice

In [82]:
spot = 41.0
strike = 40.0
vol = 0.3
rate = 0.08
div = 0.0
expiry = 1.0
steps = 3

result = EuropeanBinomPricer(callpayoff, spot, rate, vol, div, expiry, steps)
print("The price for European call option is:", result)

The price for European call option is: 7.07385326128


#### Here is the European put option

In [41]:
def EuropeanBinomPricer(putpayoff, spot, rate, vol, div, expiry, steps):
    
    h = expiry / steps
    nodes = steps + 1
    u = np.exp((rate - div) * h + vol * np.sqrt(h))
    d = np.exp((rate - div) * h - vol * np.sqrt(h))
    pstar = (np.exp((rate - div) * h) - d) / (u - d)
    discount = np.exp(-(rate - div))
    putT = 0.0
    
    for i in range(nodes):
        spotT = spot * (u ** (steps - i) * (d ** i))
        putT += putpayoff(spotT, strike) * binom.pmf(steps - i, steps, pstar)
    putprice = putT * discount
    
    
    return putprice

In [42]:
spot = 41.0
strike = 40.0
vol = 0.3
rate = 0.08
div = 0.0
expiry = 1.0
steps = 3

result1 = EuropeanBinomPricer(putpayoff, spot, rate, vol, div, expiry, steps)
print("The price for European put option is:", result1)

The price for European put option is: 2.99850711674


**Problem 2.**

In this problem, building off of the code above, you will implement the American Binomial Option Pricing Model (ABOPM). Reprice both of the options above as American options. Outline the differences between the European and American solutions.

#### Here is the American Call Option

In [114]:
def AmericanBinomPricer(callpayoff, spot, rate, vol, div, expiry, steps):
    nodes = steps + 1
    dt = expiry / steps 
    u = np.exp(((rate - div) * dt) + vol * np.sqrt(dt)) 
    d = np.exp(((rate - div) * dt) - vol * np.sqrt(dt))
    pu = (np.exp((rate - div) * dt) - d) / (u - d)
    pd = 1 - pu
    disc = np.exp(-rate * dt)
    dpu = disc * pu
    dpd = disc * pd

    Cashflowt = np.zeros(nodes)
    Spott = np.zeros(nodes)

    for i in range(nodes):
        Spott[i] = spot * (u ** (steps - i)) * (d ** i)
        Cashflowt[i] = callpayoff(Spott[i], strike)

    for i in range((steps - 1), -1, -1):
        for j in range(i+1):
            Cashflowt[j] = dpu * Cashflowt[j] + dpd * Cashflowt[j+1]
            Spott[j] = Spott[j] / u
            Cashflowt[j] = np.maximum(Cashflowt[j], callpayoff(Spott[j], strike))
            
        
    return Cashflowt[0]

In [115]:
spot = 41.0
strike = 40.0
vol = 0.3
rate = 0.08
div = 0.0
expiry = 1.0
steps = 3

result2 = AmericanBinomPricer(callpayoff, spot, rate, vol, div, expiry, steps)
print("The price for American call option is:", result2)

The price for American call option is: 7.07385326128


#### Here is the American put option

In [116]:
def AmericanBinomPricer(putpayoff, spot, rate, vol, div, expiry, steps):
    nodes = steps + 1
    dt = expiry / steps 
    u = np.exp(((rate - div) * dt) + vol * np.sqrt(dt)) 
    d = np.exp(((rate - div) * dt) - vol * np.sqrt(dt))
    pu = (np.exp((rate - div) * dt) - d) / (u - d)
    pd = 1 - pu
    disc = np.exp(-rate * dt)
    dpu = disc * pu
    dpd = disc * pd

    Cashflowt = np.zeros(nodes)
    Spott = np.zeros(nodes)

    for i in range(nodes):
        Spott[i] = spot * (u ** (steps - i)) * (d ** i)
        Cashflowt[i] = putpayoff(Spott[i], strike)

    for i in range((steps - 1), -1, -1):
        for j in range(i+1):
            Cashflowt[j] = dpu * Cashflowt[j] + dpd * Cashflowt[j+1]
            Spott[j] = Spott[j] / u
            Cashflowt[j] = np.maximum(Cashflowt[j], putpayoff(Spott[j], strike))
            
        
    return Cashflowt[0]

In [117]:
spot = 41.0
strike = 40.0
vol = 0.3
rate = 0.08
div = 0.0
expiry = 1.0
steps = 3

result2 = AmericanBinomPricer(putpayoff, spot, rate, vol, div, expiry, steps)
print("The price for American put option is:", result2)

The price for American put option is: 3.29294758539


**Problem 3.**

In this proble, again utilizing the code written above, you will implement the Naive Monte Carlo Option Pricing Model for European puts and calls. Reprice both options above.

Recall, that if we can simulate prices under the risk-neutral density, then we can statistically estimate the option price as follows:

$$
\hat{C_{0}} = \frac{1}{M} \sum\limits_{j=1}^{M} C_{T,j}
$$

where $C_{T,j}$ is the option payoff at $t=T$ for the $j^{th}$ simulated spot price. This applies to both puts and calls, so again you can use polymorphism. 

Recall, that the first model that we will use to simulate prices in discrete time was called discrete-time Brownian motion, and can be implemented as follows:

$$
S_{t + h} = S_{t} \exp{ \left((r - \delta - \frac{1}{2} \sigma^{2}) + \sigma \sqrt{h} Z \right) }
$$

where $Z$ is a draw from a standard Normal distribution (i.e. $Z \sim N(0,1)$). And in the simple Black-Scholes world we set $t + h = T$. 

This gives us the estimated mean option price. We can also calculate the estimated standard error as follows:

$$
SE(\hat{C_{0}}) = \frac{SD(C_{0,j})}{\sqrt{M}}
$$

where $SD(C_{0,j})$ is the sample standard deviation, and is given as follows:

$$
SD(C_{0.j}) = \sqrt{\frac{1}{M-1} \sum\limits_{j=1}^{M} (C_{0,j} - \hat{C_{0}})^{2}}
$$

where $C_{0,j} = \exp{(-(r - \delta) T)} C_{T,j}$ is just the present value of the option payoff for simulated spot price $j$.

Report both $\hat{C_{0}}$ and $SE(\hat{C_{0}})$.

#### Here is the European call option via Naive Monte Carlo pricer.

In [133]:
def NaiveMonteCarloPricer(callpayoff, spot, rate, vol, div, expiry, nreps):
    dt = expiry
    nudt = (rate - div - 0.5 * vol * vol) * dt
    sigdt = vol * np.sqrt(dt)
    z = np.random.normal(size=nreps)

    spott = 0.0
    cashflowt = 0.0

    spott = spot * np.exp(nudt + sigdt * z)
    cashflowt = callpayoff(spott, strike)

    price = cashflowt.mean() * np.exp(-rate * expiry)
    stderr = cashflowt.std() / np.sqrt(nreps)

    return (price, stderr)
    
    

In [136]:
spot = 41.0
strike = 40.0
expiry = 1.0
rate = 0.08
vol = 0.30
div = 0
expiry = 1
nreps = 50000

resultnaive = NaiveMonteCarloPricer(callpayoff, spot, rate, vol, div, expiry, nreps)
print("The European call price via Naive Monte Carlo Pricer is 6.9740 and the standard error is 0.04810")


The European call price via Naive Monte Carlo Pricer is 6.9740 and the standard error is 0.04810


#### Here is the European put option via Naive Monte Carlo pricer.

In [137]:
def NaiveMonteCarloPricer(putpayoff, spot, rate, vol, div, expiry, nreps):
    dt = expiry
    nudt = (rate - div - 0.5 * vol * vol) * dt
    sigdt = vol * np.sqrt(dt)
    z = np.random.normal(size=nreps)

    spott = 0.0
    cashflowt = 0.0

    spott = spot * np.exp(nudt + sigdt * z)
    cashflowt = putpayoff(spott, strike)

    price = cashflowt.mean() * np.exp(-rate * expiry)
    stderr = cashflowt.std() / np.sqrt(nreps)

    return (price, stderr)
    
    

In [142]:
spot = 41.0
strike = 40.0
expiry = 1.0
rate = 0.08
vol = 0.30
div = 0
expiry = 1
nreps = 50000

resultnaive1 = NaiveMonteCarloPricer(putpayoff, spot, rate, vol, div, expiry, nreps)
resultnaive1
print("The European put price via Naive Monte Carlo is 2.9205 and the standard error is 0.0223")


The European put price via Naive Monte Carlo is 2.9205 and the standard error is 0.0223


**Problem 4.**

In this problem you will write object-oriented code to simulate the ***Deck of Cards Workout*** from Ross Enamait's book *Never Gymless*. 

The details are as given as follows from Enamait's book:

This workout requires a standard deck of 52 playing cards. Each red card (diamonds and hearts) will require a set of burpees. Each black card (spades and clubs) will require a set of pushups. 

To perform the workout, start with a fully shuffled deck of cards. All face cards (Jacks, Queen, and King) have a value of 10. Aces have a value of 11. Number cards will be face value (ex. 7 of spades = 7 pushups). Do not use Joker cards for this problem.

Each card requires a set of pushups or burpees, depending on the color of the card. Strive to work through the entire deck as fast as possible. If burpees become too difficult, switch to squat thrusts. 

**Deck of Cards**

- Burpees for every red card
- Pushups for every black card

You will need to create classes with the appropriate data members and methods to simulate this process. You will need classes for the deck of cards, for the different suites of cards (diamonds, hearts, spades, clubs). You will need methods for drawing from the deck without replacement (*hint:* represent the deck as a list of cards, and use a pseudo-random number to sample randomly and the *pop* method to pull it from the list). You will also want a method that can randomly shuffle the deck, etc. Think carefully about this and plan it out before you begin to write any code. 

#### Here is my original idea for this question.
The orignial idea is to setup three classes. Each of the class will represent numbers of card, color or types of card, and ways to do exercise. Then I can call each of the class to achieve the goal.

In [146]:
import random


class number(object):
    def __init__(self, numbners):
        numbers = random.randint(1,14)
        return self.numbers
class color(object):
    def __init__(self, colors):
        colors = ("Diamonds", "Hearts", "Spades", "Clubs")
        return self.colors
        
class exercise(object):
    def __init__(self, ways):
        ways = ("burpees", "pushups")
        return self.ways
def HowToExercise(self, number, color):
    for i in range(colors):
        for j in range(numbers):
            print("{} of {}". format(j, i))


#### Here is what I learned from the internet.

In [151]:
class card(object):
    def __init__(self, suit, val):
        self.suit = suit
        self.value = val
    
    def show(self):
        print("{} of {}".format(self.value, self.suit))
    
class deck(object):
    def __init__(self):
        self.cards = []
        self.build()
        
    def build(self):
        for s in ["Spades", "Clubs", "Diamonds", "Hearts"]:
            for v in range(1, 14):
                self.cards.append(card(s, v))
                
    def show(self):
        for c in self.cards:
            c.show()
    def shuffle(self):
        for i in range(len(self.cards)-1, 0, -1):
            r = random.randint(0, i)
            self.cards[i], self.cards[r] = self.cards[r], self.cards[i]
        
    def drawCard(self):
        return self.cards.pop()

class Player(object):
    def __init__(self, name):
        self.name = name
        self.hand = []
        
    def draw(self, deck):
        self.hand.append(deck.drawCard())
        return self
    def showHand(self):
        for card in self.hand:
            card.show()
            
        def discard(self):
            return self.hand.pop()
        
deck.shuffle()

draw(deck).draw(deck)

NameError: name 'cards' is not defined